# Homework 08
This homework is based on the clustering lectures. Check the lecture notes and TA notes - they should help!

In [30]:
#| warning: false
#| message: false
library(dplyr)
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.1     ✔ readr     2.1.5
✔ ggplot2   4.0.0     ✔ stringr   1.5.2
✔ lubridate 1.9.4     ✔ tibble    3.3.0
✔ purrr     1.1.0     ✔ tidyr     1.3.1
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


## Question 1
This question will walk you through creating your own `kmeans` function.

#### a) What are the steps of `kmeans`?
**Hint**: There are 4 steps/builder functions that you'll need.

The steps of kmeans are as follows: 1) assign points to clusters at random, 2) compute cluster means (centroids), 3) iterate and reassign labels, 4) recalculate means.

#### b) Create the builder function for step 1.

In [17]:
label_randomly <- function(n_points, n_clusters) {
  sample(((1:n_points) %% n_clusters) + 1, n_points, replace=F)
}

#### c) Create the builder function for step 2.

In [18]:
get_cluster_means <- function(data, labels) {
  data %>%
    mutate(label__ = labels) %>%
    group_by(label__) %>%
    summarize(across(everything(), mean), .groups = "drop") %>%
    arrange(label__)
}

#### d) Create the builder function for step 3.
*Hint*: There are two ways to do this part - one is significantly more efficient than the other. You can do either.  

In [19]:
assign_cluster_fast <- function(data, means) {
  data_matrix <- as.matrix(data)
  means_matrix <- as.matrix(means %>% dplyr::select(-label__))
  dii <- sort(rep(1:nrow(data), nrow(means)))
  mii <- rep(1:nrow(means), nrow(data))
  data_repped <- data_matrix[dii, ]
  means_repped <- means_matrix[mii, ]
  diff_squared <- (data_repped - means_repped) ^ 2
  all_distances <- rowSums(diff_squared)
  tibble(dii = dii, mii = mii, distance = all_distances) %>%
    group_by(dii) %>%
    arrange(distance) %>%
    filter(row_number() == 1) %>%
    ungroup() %>%
    arrange(dii) %>%
    pull(mii)
}

assign_cluster_slow <- function(data, means) {
  dii <- 1:nrow(data)
  cii <- 1:nrow(means)
  labels <- c()
  for(point_index in dii) {
    smallest_dist <- Inf
    smallest_label <- NA
    for (clus_index in cii) {
      point <- data[point_index, ]
      clus <- means %>% dplyr::select(-label__) %>% `[`(clus_index, )
      diff <- point - clus
      dist <- sum(diff * diff)
      if (dist < smallest_dist) {
        smallest_dist <- dist
        smallest_label <- means[clus_index, ]$label__
      }
    }
    labels <- c(labels, smallest_label)
  }
  labels
}

#### e) Create the builder function for step 4.

In [20]:
kmeans_done <- function(old_means, new_means, eps=1e-6) {
  om <- as.matrix(old_means)
  nm <- as.matrix(new_means)
  m <- mean(sqrt(rowSums(om - nm) ^ 2))
  if (m < eps) TRUE else FALSE
}

#### f) Combine them all into your own `kmeans` function.

In [24]:
mykmeans <- function(data, n_clusters, eps=1e-6, max_it = 1000, verbose = FALSE) {
  labels <- label_randomly(nrow(data), n_clusters)
  old_means <- get_cluster_means(data, labels)
  done <- FALSE
  it <- 0
  while(!done & it < max_it) {
    labels <- assign_cluster_fast(data, old_means)
    new_means <- get_cluster_means(data, labels)
    if (kmeans_done(old_means, new_means)) {
      done <- TRUE
    }
    else {
      old_means <- new_means
      it <- it + 1
      if (verbose) {
        cat(sprintf("%d\n", it))
      }
    }
  }
  list(labels=labels, means=new_means)
}

## Question 2
This is when we'll test your `kmeans` function.
#### a) Read in the `voltages_df.csv` data set.

In [25]:
voltages_df <- read.csv("/content/voltages_df.csv")
head(voltages_df)

,X0,X1.00401606425703,X2.00803212851406,X3.01204819277108,X4.01606425702811,X5.02008032128514,X6.02409638554217,X7.0281124497992,X8.03212851405623,X9.03614457831325,⋯,X240.963855421687,X241.967871485944,X242.971887550201,X243.975903614458,X244.979919678715,X245.983935742972,X246.987951807229,X247.991967871486,X248.995983935743,X250
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,-1.031463,1.104665,0.8982475,0.4142208,-1.1490888,-1.078510,-1.002401,-0.9182083,-0.8215574,-0.7023741,⋯,-0.7392703,-0.7633694,-0.7792297,-0.7844340,-0.7779820,-0.7608812,-0.7369830,-0.7138199,-0.7014771,-0.7056029
2,-1.031463,1.246157,1.0948587,0.9039343,0.4654410,-1.160496,-1.112005,-1.0721319,-1.0385633,-1.0075872,⋯,-0.8859964,-0.8511675,-0.8064307,-0.7534558,-0.6954785,-0.6404759,-0.6105817,-0.6348313,-0.6767121,-0.7140939
3,-1.031463,1.216111,1.0557873,0.8417629,-0.5636836,-1.147653,-1.101783,-1.0645681,-1.0336197,-1.0051885,⋯,-0.9503509,-0.9122991,-0.8625269,-0.8016142,-0.7306757,-0.6527186,-0.5812047,-0.5875560,-0.6768023,-0.7206992
4,-1.031463,1.166244,0.9899628,0.7230858,-1.1806746,-1.125106,-1.077167,-1.0370309,-1.0027385,-0.9709488,⋯,-0.9498509,-0.9236047,-0.8896604,-0.8502120,-0.8086367,-0.7700917,-0.7418958,-0.7315532,-0.7409824,-0.7644406
5,-1.031463,1.230222,1.0746700,0.8733880,0.2116394,-1.153728,-1.106832,-1.0691075,-1.0383350,-1.0108187,⋯,-0.8710166,-0.8237315,-0.7590005,-0.6698582,-0.5061566,1.0975578,0.9348933,0.6673692,-1.1669718,-1.1047735
6,-1.031463,1.257650,1.1112886,0.9322788,0.6045620,-1.166325,-1.113488,-1.0696859,-1.0332517,-1.0009558,⋯,-0.9092342,-0.8715416,-0.8213803,-0.7589597,-0.6842115,-0.5958772,-0.4766488,1.1008087,0.9169321,0.5137345


#### b) Call your `kmeans` function with 3 clusters. Print the results with `results$labels` and `results$means`.

In [26]:
results <- mykmeans(voltages_df, 3)
print(results$labels)
print(results$means)

  [1] 3 2 2 2 2 2 1 1 2 3 3 3 1 3 2 3 3 3 3 3 2 2 2 2 2 3 3 3 1 1 1 1 2 1 1 2 2
 [38] 3 1 2 3 3 3 1 2 1 1 2 1 1 2 1 3 3 1 3 2 3 1 2 3 3 2 3 1 1 2 1 3 1 3 2 1 1
 [75] 2 2 3 1 1 1 1 1 2 1 1 2 2 2 3 1 1 1 2 1 2 2 1 2 2 1 1 1 2 1 3 3 1 3 3 2 3
[112] 3 2 1 3 1 3 3 2 2 2 2 3 3 1 2 2 1 3 1 2 3 2 1 3 2 2 2 3 1 3 2 2 1 3 3 2 3
[149] 3 1 3 1 2 2 1 3 1 1 2 3 1 1 3 2 3 1 2 3 2 3 3 1 2 3 1 1 2 1 3 2 1 2 1 3 1
[186] 2 2 3 2 3 3 2 3 3 1 1 3 2 3 3 1 3 1 1 2 3 2 1 2 3 2 1 3 1 1 2 1 1 2 2 2 2
[223] 2 2 2 1 1 1 1 3 2 3 1 2 2 1 1 3 3 1 1 3 1 1 2 1 3 1 1 2 1 2 3 1 2 1 1 3 3
[260] 3 2 2 1 3 2 2 2 1 1 1 3 2 2 2 1 3 1 1 2 2 1 3 1 3 3 2 1 2 2 1 1 3 1 1 3 2
[297] 3 1 1 2 2 1 3 3 3 1 3 2 2 2 3 3 3 3 3 3 2 3 2 3 3 1 2 1 3 3 3 3 2 3 2 3 1
[334] 3 2 2 1 1 3 2 1 2 1 1 3 1 2 1 3 3 1 2 2 2 3 1 2 3 1 1 2 2 2 2 3 1 3 2 3 3
[371] 3 3 3 2 2 1 1 3 3 3 3 1 1 3 3 1 1 2 2 1 3 2 2 2 2 1 2 2 1 3 3 3 3 2 2 3 3
[408] 1 1 1 2 1 1 3 2 2 2 1 2 2 2 3 3 2 3 1 2 2 2 2 1 1 2 1 3 2 3 3 3 1 1 1 1 3
[445] 2 3 3 3 1 2 3 3 3 2 2 3 2 2 3 1 2 

#### c) Call R's `kmeans` function with 3 clusters. Print the results with `results$labels` and `results$cluster`.
*Hint*: Use the `as.matrix()` function to make the `voltages_df` data frame a matrix before calling `kmeans()`.

In [31]:
volt_mat <- as.matrix(voltages_df)
results_2 <- kmeans(volt_mat, centers = 3)

print(results_2$centers)
print(results_2$cluster)

         X0 X1.00401606425703 X2.00803212851406 X3.01204819277108
1 -1.031463          1.091050         0.9272281         0.6317991
2 -1.031463          1.308842         1.1611321         0.9780889
3 -1.031463          1.309799         1.1622152         0.9794019
  X4.01606425702811 X5.02008032128514 X6.02409638554217 X7.0281124497992
1        -0.4083829         -1.105430         -1.043247       -0.9690121
2         0.6505168         -1.164688         -1.119307       -1.0586754
3         0.6458139         -1.172479         -1.119914       -1.0595114
  X8.03212851405623 X9.03614457831325 X10.0401606425703 X11.0441767068273
1        -0.8615512        -0.7841859        -0.6098447        -0.1658169
2        -0.9937832        -0.9231030        -0.8450179        -0.7564090
3        -0.9948450        -0.9243759        -0.8464795        -0.7580061
  X12.0481927710843 X13.0522088353414 X14.0562248995984 X15.0602409638554
1       -0.09916248        -0.2294881        -0.8766637        -0.9265620


#### d) Are your labels/clusters the same? If not, why? Are your means the same?

Both 2b and 2c functions/methods produce similar overall clustering and labeling patterns. However, the exact results for means are not identical/same. This is due to the function I created using random initial centroids which results in cluster numbering and assignment changes each time the code is run. The different random starting points can result in  different final centroids and they may not converge to the same exact minimum.

## Question 3
#### a) Explain the process of using a for loop to assign clusters for kmeans.

The process of using a loop to assign clusters is known as the slow approach which required nested iteration. For each data point , you loop over all clusters. For every cluster mean, you can compute the distance between the data point and the cluster mean. The point is assigned to the cluster that yielded the smallest distance. The process requires a double loop.

#### b) Explain the process of vectorizing the code to assign clusters for kmeans.

The process of using vectorizing to assign clusters is known as the fast approach which requires constructing 2 large replicated matrices to compare all point-clusters pairs simultaneously. This removes the need for looping.

#### c) State which (for loops or vectorizing) is more efficient and why.

Vevtorizing is a more efficient method as it uses code optimization by avoiding loops which slows down runtime (as each iterations is executed sequentially).

## Question 4
#### When does `kmeans` fail? What assumption does `kmeans` use that causes it to fail in this situation?

Kmeans failing can be a result of the clusters not being roughly spherical or equally sized, clusters can have differing densities or variances. Other reasons include data not being in the vector space or there existing extreme outliers in the data. kmeans uses the assumptions that we are dealing with vectorial data, clusters are spherical, and each point only belongs to one cluster.

## Question 5
#### What assumption do Guassian mixture models make?

Guassian mixture models assume that the data is generated by a collection of Guassian distributions, each of which represent a cluster. Each cluster has its own mean and covariance. This is different from kmeans as it assumes each data point belongs to clusters with certain probabilities. Whereas in kmeans, it is assumed that each data point is assigned to a single cluster.


## Question 6
#### What assumption does spectral clustering make? Why does this help us?

Spectral clustering makes the assumption that two points are more likely to be in the same cluster if they are close to one another. This is used to inform the graph to connect the data set. This type of clustering relies on a similarity measure which checks distance allowing it to work for nonllinear, irregularly shaped clusters, and non-vector data. This helps us because we can handle more complex data structures with graphs for networks where relationships are defined by connectivity or similarity.

## Question 7
#### Define the gap statistic method. What do we use it for?

The gap statistic method compares the clustering for each value of K to a cluster of randomized data into the same domian with the original data. Then this dispersions of the real data and randomized data clusters are compared to then find the optimal cluster number at the knee of the plot. This method is considered ad-hoc and is used to find optimal number of clusters for a dataset.